In [38]:
from sent2vec.vectorizer import Vectorizer
from nltk.tokenize import sent_tokenize
from dataset import get_examples
from word_frequency import *
from pathlib import Path
from utils import *
import sys


#train sent2vec
# load sentence to vec mode from wiki_unigrams.bin

# Load sent2vec vectorizer
# using bert-base-uncased
# vectorizer.bert("bert-base-uncased")



In [39]:

#train sent2vec
# load sentence to vec mode from wiki_unigrams.bin

# Load sent2vec vectorizer
# using bert-base-uncased
# vectorizer.bert("bert-base-uncased")
vectorizer = Vectorizer()

# my own multi dimensional list iterator :)

def max_iterator(iterator_list, previous_index = []):
    if len(iterator_list) > 0: 
        for i in range(iterator_list[-1]):
            max_iterator(iterator_list[:-1], [i] + previous_index)
    elif len(iterator_list) == 0:
        do_thing(previous_index)
        print(previous_index)

def do_thing(previous_index, sub_list):
    sub_list.append()
            

# make this generalized
summed_percentile = 0.68
words_to_remove = list(subset_percentile(summed_percentile)["word"])
count_word_to_remove = len(words_to_remove)

print("length of words to remove", count_word_to_remove)
print("Word Vocab Space", word_space() - count_word_to_remove)


#sys.exit()

training_exaples = get_examples("train", jypter=True)




Initializing Bert distilbert-base-uncased
Vectorization done on cuda


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


length of words to remove 331013
Word Vocab Space 2320
7473 train examples


In [40]:

questions = [example["question"] for example in training_exaples]


sentences = tokenize_sentence(questions[0])
    
sentence = sentences[1]
words = filter_words(sentence) 

# remove words that are not in the data
valid_words = [x for x in words if word_in_data(x)]

word_to_remove = [words for words in words if words in words_to_remove]






In [41]:
valid_words

['how',
 'many',
 'clips',
 'did',
 'natalia',
 'sell',
 'altogether',
 'in',
 'april',
 'and',
 'may']

In [42]:
word_to_remove

['clips', 'natalia', 'altogether']

In [197]:
replacement_tuples = {}

for word in word_to_remove:
    # for word, appends valid synonym

    replacement_tuples[word] = synonym_better(word)[0]
    # dict structure is {word: [synonym, synonym, synonym]}

# make possible matrix of all possible combinations of sentences

# find where the word is in the sentence including uppercase and lowercase combinations
words = replacement_tuples.keys()

#index_start = [0 for x in range(len(words))]
print(replacement_tuples)

index_end = [len(replacement_tuples[x]) for x in words]

{'clips': [], 'natalia': [], 'altogether': ['raw', 'all', 'completely', 'entirely', 'right', 'totally', 'whole', 'wholly']}


In [196]:
index_end

[0, 0, 8]

In [45]:
index_end

[0, 0, 8]

In [163]:
def make_sentence(previous_index, sentence, replacement_tuples):
    # key, replacement_tuples[index]
    word_subs = [(replacement_tuple[0], replacement_tuple[1]) for index, replacement_tuple in enumerate(replacement_tuples.items())]
    paired_value = [(replacement_tuple[0], replacement_tuple[1][previous_index[index]]) for index, replacement_tuple in enumerate(word_subs) if replacement_tuple[1] != []]

    #print(previous_index)
    #print(paired_value)

    for word, replacement in paired_value:
        # begging and ending of a sentence
        sentence_indicies = replacement_index(sentence, word, "a")

        if sentence_indicies[0] == 0:
            sentence = replacement[:1].upper() + replacement[1:] + sentence[sentence_indicies[1]:]
        else:
            sentence = sentence[:sentence_indicies[0]] + replacement + sentence[sentence_indicies[1]:]
        
    return sentence


In [214]:
def max_iterator(iterator_list, previous_index = [], function_args = None):
    sentences = []
    if len(iterator_list) > 0: 
        # if the last index is 0, then we don't want to iterate over it and just call the function
        if (iterator_list[-1] == 0):
            sentence = max_iterator(iterator_list[:-1], [0] + previous_index, function_args)                                                                                                                                                                                                                                                              
            sentences.append(sentence)

        for i in range(iterator_list[-1]):
            sentence = max_iterator(iterator_list[:-1], [i] + previous_index, function_args)                                                                                                                                                                                                                                                              
            sentences.append(sentence)
        print(iterator_list)
        return sentences
    elif len(iterator_list) == 0:
        # start building the sentence
        return make_sentence(previous_index, function_args[0], function_args[1])



In [215]:
sentences_unflattened = max_iteratora([0,0,8], previous_index=[], function_args = [sentence, replacement_tuples])
import numpy as np

sentences_unflattened = np.asarray(sentences_unflattened)
sentences_unflattened.flatten()

[0]
[0, 0]
[0]
[0, 0]
[0]
[0, 0]
[0]
[0, 0]
[0]
[0, 0]
[0]
[0, 0]
[0]
[0, 0]
[0]
[0, 0]
[0, 0, 8]


array(['How many clips did Natalia sell raw in April and May?\n',
       'How many clips did Natalia sell all in April and May?\n',
       'How many clips did Natalia sell completely in April and May?\n',
       'How many clips did Natalia sell entirely in April and May?\n',
       'How many clips did Natalia sell right in April and May?\n',
       'How many clips did Natalia sell totally in April and May?\n',
       'How many clips did Natalia sell whole in April and May?\n',
       'How many clips did Natalia sell wholly in April and May?\n'],
      dtype='<U61')